### Exercise: Add polynomial distortion to the imaging example

In this exercise we'll add polynomial distortion in `x` and `y`.

#### Directions

##### A 2D polynomial model is instantiated using
  
  
    from astropy.modeling import models
    p = models.Polynomial2D(degree=1, c0_0=.3)
  
To see the parameter names execute

    models.Polynomial2D(degree=1).param_names

  
Generate 2 polynomials in `x` and `y`  to correct the distortion on each axis: `px` and `py`.
  
##### Generate a combine model using
  

    distortion = models.Mapping((0, 1, 0, 1) | px & py

  
##### Folowing the imaging example above add this to the WCS pipeline as a first step, creating an intermediate frame of "undistorted coordinates"

  
    from gwcs import coordinate_frames as cf
      
    undistorted = cf.Frame2D(name='undistorted', unit=(u.pix, u.pix))
  
  

In [ ]:
from astropy.modeling import models
from astropy import units as u
from astropy import coordinates as coord

import asdf
from gwcs import wcs, coordinate_frames as cf

In [ ]:
# Check the names of the parameters
models.Polynomial2D(degree=1).param_names

In [ ]:
px = models.Polynomial2D(1, c0_0=-1, c1_0=1, c0_1=1)
py = models.Polynomial2D(1, c0_0=-2, c1_0=1, c0_1=1)

distortion = models.Mapping((0, 1, 0, 1)) | px & py

In [ ]:
print(distortion(1,1))

In [ ]:
undistorted = cf.Frame2D(name='undistorted', unit=(u.pix, u.pix))

There are two ways we can complete the exercise:
- generate a new WCS object and include the new intermediate frame in it
- use the existing WCS object and insert a new intermediate frame

We'll do both as an exercise, starting with the second one.

First, let's read in the file with the WCS wwe saved before in the tutorial notebook.

In [ ]:
with asdf.open('wcs.asdf') as af:
    wcsobj = af['wcs']

print(wcsobj)

In [ ]:
wcsobj.pipeline

Instead of generating a new WCS object we can insert the frame into the old one. One of the advantages of GWCS is its flexibility. A WCS pipeline can be tweaked or adjusted after its generated.

In [ ]:
wcsobj.insert_frame?

In [ ]:
wcsobj.insert_frame(wcsobj.detector, distortion, undistorted)

In [ ]:
print(wcsobj)

In [ ]:
wcsobj(1,1)

Above the distortion model doesn't have a name and it reuses the name of the model class, in this case `CompoundModel`. To be more user friendly a name can be assigned.

In [ ]:
wcsobj.get_transform('detector', 'undistorted').name = 'distortion'
print(wcsobj)

Now save this object to a file and read it back in to show all changes we made were serialized and deserialized correctly.

In [ ]:
af = asdf.AsdfFile(tree={'wcs': wcsobj})
af.write_to('wcsnew.asdf')

In [ ]:
with asdf.open('wcsnew.asdf') as af:
    w = af['wcs']
    
    print(f"ra, dec = {w(1, 1)} \n")

    print(w)

#### Exercise 2: Generate high level objects and transform to different coordinate systems

The common WCS API methods do not accept keyword arguments. In order to generate high level objects we will fix the spectral order.

- Generate a WCS object for the same object and fix the spectral order to -1, save it to a file.
- Call the high level "pixel_to_world" method and inspect the output
- Transform the sky object to galactic coordinates
- Transform the spectral coordinate to Angstroms

In [ ]:
# open the file with the grism observation
with asdf.open('../data/grism.asdf') as agrism:
    wgrism = agrism['meta']['wcs']

In [ ]:
# fix the coordinates of the object to x0=677 and y0=1042, and the spectral order to -1.

wcs541_n1 = wgrism.fix_inputs({2:677, 3:1042, 4:-1})

af = asdf.AsdfFile(tree={'wcs': wcs541_n1}) 
af.write_to('wcs541_n1.asdf')

In [ ]:
wcs541_n1.pixel_to_world(680, 1616)

In [ ]:
sky, spec = wcs541_n1.pixel_to_world(680, 1616)
print(f"Celestial coordinate in ICRS {sky}")
print(f"Celestial coordinate in Galactic {sky.galactic}")

In [ ]:
print(spec.Angstrom)